<img src="catedra.jpg" width="400"/> 
<div style="text-align: right"></div>

<h3 id="proyectointerciclo">Inter-cycle Project</h3>

_<b>integrator</b> Soa Amendaño
_<b></b> Christian Dután 
_<b>Teacher:</b> Vladimir Robles._  

<h1 id="asimulationofdiscreteeventsofthevehiculartechnicalrevisionrtvwillbecarriedout">A simulation of discrete events of the vehicular technical revision (rtv) will be carried out</h1>

The example we are going to use is the following:

The attention is delayed between 15 and 30 minutes per vehicle, that is to say that each revision position receives on average 3 vehicles per hour, for the execution of the program we will use pseudo-random numbers to have a uniform arrival order
 
 
 <img src="rtv.jpg" width="400"/> 
<div style="text-align: right"></div>

<h1 id="simulation">Simulation</h1>

+ The time between arrivals of the vehicles to the vehicular technical review is exponential with an average of 10 minutes.
+ The vehicular technical revision is carried out in 3 cubilos which takes between 15 and 30 minutes per vehicle,
+ The vehicles are attended based on the FIFO discipline (first to arrive, first to leave).


So the arrivals are calculated according to the following formula:

<h4 id="t_llegadasxlnr">T_LLEGADAS = –x ln(R)</h4>

Where:

+ x is the average of arrivals, in our example it is 10 minutes.
+ Ln is the operation of Natural Logarithm
+ R is a pseudorandom number

The service times are calculated according to the following formula:


#### tiempo_revision = TIEMPO_REVISION_MIN + (TIEMPO_REVISION_MAX – TIEMPO_REVISION_MIN) R


This is: the minimum time that the revision is delayed, in our example it is 15, plus the difference between the maximum and the minimum, in our example it would be 15 minutes (30 minus 15), multiplied by a pseudo-random number. The result will give us a number between 15 and 30.


#### The pseudorandom numbers that we will use will be the following:



In [1]:
import random

random.seed(30)

for i in range(10):
    print('{:04.4f}'.format(random.random()), end=' ')
print()

0.5391 0.2892 0.0300 0.6536 0.2100 0.2573 0.3972 0.6416 0.9888 0.4615 


We will use the first 5 to calculate the arrivals and the last 5 to calculate the service times

## Calculation of arrival times:

+ Vehicle 1. -10 * ln (0.5391) = 6.18

+ Vehicle 2. -10 * ln (0.2892) = 12.41 (we add 6.18 the result is: 18.59)

+ Vehicle 3. -10 * ln (0.0300) = 12.04 + 18.59 = 30.63

+ Vehicle 4. -10 * ln (0.6536) = 4.25 + 30.63 = 34.88

+ Vehicle 5. -10 * ln (0.2100) = 15.61 + 34.88 = 50.49

Between minute 0 and arrival of vehicle 1, 6.18 minutes passed. Between minute 6.18 and the arrival of vehicle 2 passed 12.41 minutes, so vehicle 2 arrived at 18.59 minutes.

## Calculation of service times:

+ Vehicle 1. 15 + (15 * 0.2573) = 18.86

+ Vehicle 2. 15 + (15 * 0.3972) = 20.96

+ Vehicle 3. 15 + (15 * 0.6416) = 24.62

+ Vehicle 4. 15 + (15 * 0.9888) = 29.83

+ Vehicle 5. 15 + (15 * 0.4615) = 21.92

Vehicle 1 delayed 18.86 minutes, vehicle 2 delayed 20.96 minutes. The vehicle that most delayed was the 4, delayed 29.83 minutes. As you can see, the service time goes from 15 to 30 minutes.

In [2]:
import pandas as pd

data = pd.read_csv('Datos.csv')

data.head(6)

,Vehiculo,LLegada,Duracion_de_la_Revision,Salida,Tiempo_de_Espera
0,1,6.18,18.86,25.04,0.00
1,2,18.59,20.96,39.55,6.45
2,3,30.63,24.62,55.25,8.92
3,4,34.88,29.83,64.71,20.37
4,5,50.49,21.92,72.41,14.22
5,0,0.00,116.19,0.00,49.96


# ADEQUATE SIMULATION TO MAKE DATA COMPARISON

In [3]:
import random
import math
import simpy
import matplotlib.pyplot as pp


SEMILLA = 30
NUM_CUBICULOS = 1
TIEMPO_REVISION_MIN = 15
TIEMPO_REVISION_MAX = 30
T_LLEGADAS = 10
TIEMPO_SIMULACION = 120
TOT_VEHICULOS = 5
 
te  = 0.0 # tiempo de espera total
dt  = 0.0 # duracion de servicio total
fin = 0.0 # minuto en el que finaliza

def revision(vehiculo):
	global dt  #Para poder acceder a la variable dt declarada anteriormente
	R = random.random()  # Obtiene un numero aleatorio y lo guarda en R
	tiempo = TIEMPO_REVISION_MAX - TIEMPO_REVISION_MIN  
	tiempo_revision = TIEMPO_REVISION_MIN + (tiempo*R) # Distribucion uniforme
	yield env.timeout(tiempo_revision) # deja correr el tiempo n minutos
	print(" \o/ Revision tecnica vehicular listo a %s en %.2f minutos" % (vehiculo,tiempo_revision))
	dt = dt + tiempo_revision # Acumula los tiempos de uso de la i

    
def vehiculo (env, nombre, vehic):
	global te
	global fin
	llega = env.now # Guarda el minuto de llegada del cliente
	print ("---> %s llego a la revision en minuto %.2f" % (nombre, llega))
	with vehic.request() as request: # Espera su turno
		yield request # Obtiene turno
		pasa = env.now # Guarda el minuto cuado comienza a ser atendido
		espera = pasa - llega # Calcula el tiempo que espero
		te = te + espera # Acumula los tiempos de espera
		print ("**** %s pasa a la revision en minuto %.2f habiendo esperado %.2f" % (nombre, pasa, espera))
		yield env.process(revision(nombre)) # Invoca al 
		deja = env.now #Guarda el minuto en que termina el proceso  
		print ("<--- %s sale de la revision en minuto %.2f" % (nombre, deja))
		fin = deja # Conserva globalmente el ultimo minuto de la simulacion


def principal (env, vehic):
	llegada = 0
	i = 0
	for i in range(TOT_VEHICULOS): # Para n vehiculos
		R = random.random()
		llegada = -T_LLEGADAS * math.log(R) # Distribucion exponencial
		yield env.timeout(llegada)  # Deja transcurrir un tiempo entre uno y otro
		i += 1
		env.process(vehiculo(env, 'Vehiculos %d' % i, vehic)) 
        
        
print ("------------------- Bienvenido Simulacion REVISION TECNICA VEHICULAR ------------------")
random.seed (SEMILLA)  # Cualquier valor
env = simpy.Environment() # Crea el objeto entorno de simulacion
vehic = simpy.Resource(env, NUM_CUBICULOS) #Crea los recursos 
env.process(principal(env, vehic)) #Invoca el proceso princial
env.run() #Inicia la simulacion


print ("\n---------------------------------------------------------------------")
print ("\nIndicadores obtenidos: ")

lpc = te / fin
print ("\nLongitud promedio de la cola: %.2f" % lpc)
tep = te / TOT_VEHICULOS
print ("Tiempo de espera promedio = %.2f" % tep)
upi = (dt / fin) / NUM_CUBICULOS
print ("Uso promedio de la instalacion = %.2f" % upi)
print ("\n---------------------------------------------------------------------")


------------------- Bienvenido Simulacion REVISION TECNICA VEHICULAR ------------------
---> Vehiculos 1 llego a la revision en minuto 6.18
**** Vehiculos 1 pasa a la revision en minuto 6.18 habiendo esperado 0.00
---> Vehiculos 2 llego a la revision en minuto 18.59
 \o/ Revision tecnica vehicular listo a Vehiculos 1 en 15.45 minutos
<--- Vehiculos 1 sale de la revision en minuto 21.63
**** Vehiculos 2 pasa a la revision en minuto 21.63 habiendo esperado 3.04
---> Vehiculos 3 llego a la revision en minuto 22.84
---> Vehiculos 4 llego a la revision en minuto 36.41
 \o/ Revision tecnica vehicular listo a Vehiculos 2 en 18.15 minutos
<--- Vehiculos 2 sale de la revision en minuto 39.78
**** Vehiculos 3 pasa a la revision en minuto 39.78 habiendo esperado 16.94
---> Vehiculos 5 llego a la revision en minuto 45.65
 \o/ Revision tecnica vehicular listo a Vehiculos 3 en 24.62 minutos
<--- Vehiculos 3 sale de la revision en minuto 64.40
**** Vehiculos 4 pasa a la revision en minuto 64.40 habie

# SIMULACION BASADO EN EVENTOS REALES

 <img src="ESTADISTICA.PNG" width="800"/> 
<div style="text-align: right"></div>

In [4]:
import random
import math
import simpy
import matplotlib.pyplot as pp


SEMILLA = 30
NUM_CUBICULOS = 3
TIEMPO_REVISION_MIN = 15
TIEMPO_REVISION_MAX = 30
T_LLEGADAS = 5
TIEMPO_SIMULACION = 120
TOT_VEHICULOS = 70
 
te  = 0.0 # tiempo de espera total
dt  = 0.0 # duracion de servicio total
fin = 0.0 # minuto en el que finaliza

# Diccionario para las graficas
gte={}
gdt={}
gfin={}

def revision(vehiculo):
	global dt  #Para poder acceder a la variable dt declarada anteriormente
	R = random.random()  # Obtiene un numero aleatorio y lo guarda en R
	tiempo = TIEMPO_REVISION_MAX - TIEMPO_REVISION_MIN  
	tiempo_revision = TIEMPO_REVISION_MIN + (tiempo*R) # Distribucion uniforme
	yield env.timeout(tiempo_revision) # deja correr el tiempo n minutos
	print(" \o/ Revision tecnica vehicular listo a %s en %.2f minutos" % (vehiculo,tiempo_revision))
	dt = dt + tiempo_revision # Acumula los tiempos de uso de la i

    
def vehiculo (env, nombre, vehic):
	global te
	global fin
	llega = env.now # Guarda el minuto de llegada del cliente
	print ("---> %s llego a la revision en minuto %.2f" % (nombre, llega))
	with vehic.request() as request: # Espera su turno
		yield request # Obtiene turno
		pasa = env.now # Guarda el minuto cuado comienza a ser atendido
		espera = pasa - llega # Calcula el tiempo que espero
		te = te + espera # Acumula los tiempos de espera
		print ("**** %s pasa a la revision en minuto %.2f habiendo esperado %.2f" % (nombre, pasa, espera))
		yield env.process(revision(nombre)) # Invoca al 
		deja = env.now #Guarda el minuto en que termina el proceso  
		print ("<--- %s sale de la revision en minuto %.2f" % (nombre, deja))
		fin = deja # Conserva globalmente el ultimo minuto de la simulacion


def principal (env, vehic):
	llegada = 0
	i = 0
	for i in range(TOT_VEHICULOS): # Para n vehiculos
		R = random.random()
		llegada = -T_LLEGADAS * math.log(R) # Distribucion exponencial
		yield env.timeout(llegada)  # Deja transcurrir un tiempo entre uno y otro
		i += 1
		env.process(vehiculo(env, 'Vehiculos %d' % i, vehic)) 
        
        
print ("------------------- Bienvenido Simulacion REVISION TECNICA VEHICULAR ------------------")
random.seed (SEMILLA)  # Cualquier valor
env = simpy.Environment() # Crea el objeto entorno de simulacion
vehic = simpy.Resource(env, NUM_CUBICULOS) #Crea los recursos 
env.process(principal(env, vehic)) #Invoca el proceso princial
env.run() #Inicia la simulacion


print ("\n---------------------------------------------------------------------")
print ("\nIndicadores obtenidos: ")

lpc = te / fin
print ("\nLongitud promedio de la cola: %.2f" % lpc)
tep = te / TOT_VEHICULOS
print ("Tiempo de espera promedio = %.2f" % tep)
upi = (dt / fin) / NUM_CUBICULOS
print ("Uso promedio de la instalacion = %.2f" % upi)
print ("\n---------------------------------------------------------------------")


------------------- Bienvenido Simulacion REVISION TECNICA VEHICULAR ------------------
---> Vehiculos 1 llego a la revision en minuto 3.09
**** Vehiculos 1 pasa a la revision en minuto 3.09 habiendo esperado 0.00
---> Vehiculos 2 llego a la revision en minuto 9.29
**** Vehiculos 2 pasa a la revision en minuto 9.29 habiendo esperado 0.00
---> Vehiculos 3 llego a la revision en minuto 11.42
**** Vehiculos 3 pasa a la revision en minuto 11.42 habiendo esperado 0.00
---> Vehiculos 4 llego a la revision en minuto 18.21
 \o/ Revision tecnica vehicular listo a Vehiculos 1 en 15.45 minutos
<--- Vehiculos 1 sale de la revision en minuto 18.54
**** Vehiculos 4 pasa a la revision en minuto 18.54 habiendo esperado 0.33
---> Vehiculos 5 llego a la revision en minuto 20.43
---> Vehiculos 6 llego a la revision en minuto 24.29
---> Vehiculos 7 llego a la revision en minuto 24.32
---> Vehiculos 8 llego a la revision en minuto 24.36
 \o/ Revision tecnica vehicular listo a Vehiculos 2 en 18.15 minutos
<